In [2]:
#edasiarendus clean_code_loincing2.py
import psycopg2 as pg2
import os
import pandas as pd
import sys
from sqlalchemy import create_engine

sys.path
sys.path.append('/home/anneott/Repos/cda-data-cleaning/analysis_data_cleaning/LOINC_cleaning')

import cleaning_functions as cf
import import_data as data

Huvipakkuvad veerud ja andmestiku sisselugemine andmebaasi/käsitsi/csv kaudu

In [6]:
columns = ['epi_id', 'analysis_name', 'parameter_code', 'parameter_name', 'unit']

#kontrolli, et querys oleks igal templatel unikaalsed read
#kõik vere alla kuuluvad templated
#failist blood_templates.sql
query = "select epi_id, analysis_name, parameter_code_raw_entry as parameter_code,  parameter_name, parameter_unit as unit from  egcut_epi.analysiscleaning.r07_analysis_matched where (epi_id, analysis_name) in (select epi_id, analysis_name from ( select epi_id, analysis_name, array_agg(distinct parameter_name) as param_hulk, row_number() over (partition by array_agg(distinct parameter_name)) as reanumber from egcut_epi.analysiscleaning.r07_analysis_matched where analysis_name in (select analysis_name from egcut_epi.analysiscleaning.analysis_classified where system_type = 'blood') group by epi_id, analysis_name) as tabel1 where reanumber = 1 and array_length(param_hulk,1) > 1) group by epi_id, analysis_name, parameter_code, parameter_name, unit having count(*) = 1 limit 10;"
blood = data.get_dataframe_from_db(query)
#blood

Lisab uued veerud süsteemi tüübi ja analüüdiga

In [14]:
df = cf.add_bloodanalytes_and_systemtype(blood, columns)
#df

IgD


Unitite parandamine ja propertyte määramine

In [15]:
df = cf.clean_units(df)
df = cf.add_property(df)
df = cf.apply_property_rules(df) 
#df

Splitting_bloodi reeglite rakendamine, süsteemi tüübi jagamine süsteemideks ja tühjade väljade muutmine None'ks;
Muudab analyte veeru nime systemiks

In [16]:
blood_classes = cf.create_blood_classes(blood, columns)
#blood_classes
#tekivad duplikaadid SerPlasiga

PRAHT


Andmete kokkupanek

In [17]:
merged_dataframe = df.merge(blood_classes, how='left',
                                    on=['epi_id','analysis_name', 'parameter_code', 'parameter_name', 'unit',
                                        'system_type']
                                    , indicator=True
                                    )

merged_dataframe.loc[merged_dataframe.system.isnull() & (merged_dataframe.system_type == 'blood'), 'system'] = 'Bld'
#merged_dataframe

Jätame alles ainult loinci mappimiseks vajalikud veerud

In [18]:
loinc_mapping = merged_dataframe[['epi_id', 'analysis_name','parameter_code','parameter_name','unit','system_type','analyte','property','system']]
#loinc_mapping

Tabel tagasi datagripi

In [5]:
connection_string = "host='p12.stacc.ee' dbname='egcut_epi' user='username' password='pw'"
engine = create_engine('postgresql://username:pw@p12.stacc.ee:5432/egcut_epi', connect_args={'options': '-csearch_path=analysiscleaning'})
print('tehtud')

tehtud


NameError: name 'password' is not defined

In [20]:
loinc_mapping.to_sql('mapped_blood_templates', engine)
print('tehtud')

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "username"
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
#loinc_mapping.to_csv('loinc_mapping_hemogramm5leuko_templates.csv')
#print('tehtud')